To run

```sh
git clone <this branch>
cd diffusers
python -m venv ./venv
source .venv/bin/activate
pip install -U pip wheel
pip install -e .[torch]
pip install --upgrade transformers scipy
pip install jupyterlab moviepy
````


In [ ]:
import torch
from torch import Generator
from diffusers import DiffusionPipeline
import moviepy.editor as mpy
import numpy as np
import tqdm

torch.set_grad_enabled(False)
device = torch.device("cuda")
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1"
)
pipe.to(device)
pass

In [ ]:
# ?pipe

In [ ]:
print(type(pipe))
print(type(pipe.text_encoder))
print(type(pipe.unet))
print(type(pipe.vae))
print(type(pipe.scheduler))

In [ ]:
generator = torch.Generator(device=device)
generator.manual_seed(1)

tape = []

@torch.no_grad()
def callback(i, t, latents):
    latents = latents.detach().clone()
    instant = (i, t, latents)
    tape.append(instant)

prompt = ["A photo of a white robot in tall grass with staring at clouds in the blue sky"]
result = pipe(
    prompt=prompt,
    num_inference_steps=50,
    num_images_per_prompt=3,
    callback=callback,
    generator=generator,
    guidance_scale=7.5,
)
for image in result.images:
    display(image)

In [ ]:
image_seq = []
for (i, t, latents) in tqdm.tqdm(tape):
    images = pipe.decode_latents(latents)
    images = (images * 255).astype(np.uint8)
    image = np.concatenate(images, axis=1)
    image_seq.append(image)

In [ ]:
clip = mpy.ImageSequenceClip(image_seq + 70 * image_seq[-1:], fps=20)
clip.write_videofile("/tmp/vid.mp4")
# clip.write_gif("/tmp/vid.gif")
!gifski /tmp/vid.mp4 --width 1200 -o /tmp/vid.gif